# Creamos dos modelos pésimos

In [1]:
import pandas as pd
import pickle
import json
import requests
import random
import time
import sqlite3

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression


In [2]:
X = load_wine(return_X_y=False, as_frame=True)

In [3]:

# Este renaming es necesario para no romper el json, no pregunten como lo se, algún "DATA SCIENTIST" de por ahí tardé 3 horas en darme cuenta

X['data'].rename(columns={'od280/od315_of_diluted_wines':'od280'}, inplace=True)

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [4]:
rf = RandomForestClassifier(random_state=42, n_estimators=10)
rf.fit(X['data'][X['data'].index.isin([100,177])], X['target'][X['target'].index.isin([100,177])])

RandomForestClassifier(n_estimators=10, random_state=42)

In [5]:
y_pred = rf.predict(X['data'])
print(classification_report(X['target'], y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.45      0.97      0.62        71
           2       0.92      0.48      0.63        48

    accuracy                           0.52       178
   macro avg       0.46      0.48      0.42       178
weighted avg       0.43      0.52      0.42       178

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [6]:
confusion_matrix(X['target'], y_pred) 

array([[ 0, 59,  0],
       [ 0, 69,  2],
       [ 0, 25, 23]])

In [7]:
lr = LogisticRegression(random_state=42)
lr.fit(X['data'][X['data'].index.isin([0,177])], X['target'][X['target'].index.isin([0,177])])

LogisticRegression(random_state=42)

In [8]:
y_pred_lr = lr.predict(X['data'])
print(classification_report(X['target'], y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88        59
           1       0.00      0.00      0.00        71
           2       0.37      0.90      0.52        48

    accuracy                           0.54       178
   macro avg       0.41      0.60      0.47       178
weighted avg       0.38      0.54      0.43       178

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

In [9]:
confusion_matrix(X['target'], y_pred_lr) 

array([[53,  0,  6],
       [ 4,  0, 67],
       [ 5,  0, 43]])

In [10]:

# Pickle y papitas a todo

pickle.dump(rf, open('saved_models/rf_model_01.pkl', 'wb'))

pickle.dump(lr, open('saved_models/lr_model_01.pkl', 'wb'))

pickle.dump(list(X['data'].columns), open('saved_models/columns_01.pkl', 'wb'))


# Como correr el API desde el entorno local

![Como activar todo](Imagenes/image-20221102-202816.png)

# Como correr el API desde Deepnote

Básicamente tenemos que activar una funcionalidad y estamos. Vamos a ir a la configuración del environment y setear que acepte conexiones, una vez hecho, copiamos y pegamos la url y estamos!

![Activar conexión](Imagenes/activar_conexion_externa.gif)

In [11]:
url_deepnote = 'https://d0186863-7f19-4501-966b-e3e8527212f8.deepnoteproject.com'

print(f'URL login dashboard: \n {url_deepnote}/dashboard/login')

URL login dashboard: 
 https://d0186863-7f19-4501-966b-e3e8527212f8.deepnoteproject.com/dashboard/login


Para ejecutar desde Deepnote necesitan abrir una terminal(abajo a la derecha está la opción) y hacer lo mismo que hicieron localmente, algo así (yo cambié el directorio porque quedó todo medio desacomodado):


![Picture title](Imagenes/image-20221102-204702.png)

# Para testear desde Deepnote

In [45]:

response = requests.post(f"{url_deepnote}/predict/saved_models/v1/rf_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [200]>
prediccion:  {'model': 'rf_01', 'prediction': '[1, 2]'}


In [26]:
print(f"{url_deepnote}/predict/saved_models/v1/lr_01")

https://d0186863-7f19-4501-966b-e3e8527212f8.deepnoteproject.com/predict/saved_models/v1/lr_01


In [40]:

mi_nombre = "Tomy"

response = requests.post(f"{url_deepnote}/bienvenida/{mi_nombre}")

response.text


'Hola Tomy! viste que tremenda API? (by the way usaste el método POST)'

# Predicciones pre-calculadas

In [32]:
# Vamos a guardar las predicciones como parquet

pd.DataFrame({'ID': X['data'].index, 'PREDS': lr.predict(X['data'])}).to_parquet('PREDS/lr_01.parquet')
pd.DataFrame({'ID': X['data'].index, 'PREDS': rf.predict(X['data'])}).to_parquet('PREDS/rf_01.parquet')

pd.read_parquet('PREDS/rf_01.parquet').head(3)

,ID,PREDS
0,0,1
1,1,1
2,2,1


In [51]:
response = requests.post(f"{url_deepnote}/predict/saved_preds/v1/lr_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [404]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Para testear desde Postman a mano

In [19]:
## para testear en postman copian y pegan el print en el request
final = json.dumps([X['data'].loc[90].to_dict(), X['data'].loc[0].to_dict()], indent=2)
print(final)

[
  {
    "alcohol": 12.08,
    "malic_acid": 1.83,
    "ash": 2.32,
    "alcalinity_of_ash": 18.5,
    "magnesium": 81.0,
    "total_phenols": 1.6,
    "flavanoids": 1.5,
    "nonflavanoid_phenols": 0.52,
    "proanthocyanins": 1.64,
    "color_intensity": 2.4,
    "hue": 1.08,
    "od280": 2.27,
    "proline": 480.0
  },
  {
    "alcohol": 14.23,
    "malic_acid": 1.71,
    "ash": 2.43,
    "alcalinity_of_ash": 15.6,
    "magnesium": 127.0,
    "total_phenols": 2.8,
    "flavanoids": 3.06,
    "nonflavanoid_phenols": 0.28,
    "proanthocyanins": 2.29,
    "color_intensity": 5.64,
    "hue": 1.04,
    "od280": 3.92,
    "proline": 1065.0
  }
]


Nota al pie: Postman es un app que tiene algunas cosas interesantes para probar APIs, recomendable!



![Picture title](Imagenes/image-20221102-210544.png)



Acá se ve que usé GET en vez de POST (miren el código y van a ver que permití ambos aunque no hice ningún comportamiento distinto

![Imagencita](Imagenes/image-20221102-210607.png)

# Esto es para mandar request desde la ejecución local

Consideren que pueden pegarle al API abierta cambiando la parte que viene antes de la ruta (reemplazan lo que está antes del "/predict/..." y le pegan local al API abierta desde Deepnote

In [28]:

response = requests.post("http://127.0.0.1:12345/predict/saved_model/v1/lr_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=12345): Max retries exceeded with url: /predict/saved_model/v1/lr_01 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9207e7c460>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
response = requests.post("http://127.0.0.1:12345/predict/saved_model/v1/rf_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [200]>
prediccion:  {'model': 'rf_01', 'model_': 'rf_01', 'model__': 'rf_01', 'prediction': '[1, 2]'}


In [ ]:

# Para "simular" pedidos a la API podemos ejecutar esta celda

for i in range(200):
    time.sleep(random.randint(1,3))
    model_ = random.choice(['rf_01', 'lr_01'])
    response = requests.post(f"http://127.0.0.1:12345/predict/v1/{model_}", json=[X['data'].sample(random.randint(0, len(X['data']))).to_dict()])

# Monitoreo desde Flask Dashboard

El monitoreo es básico en serio, solo dice como le fue a la API desde su lado así que no tiene mucha información pero es divertido ver el mapita de calor. 

Algo interesante es que genera una base de datos SQLite que podemos consultar y mirar lo que tiene adentro. El esquema está en https://flask-monitoringdashboard.readthedocs.io/en/latest/developing.html#architecture


El monitoreo se abre desde url:puerto que estés usando + "/dashboard/login" y si no tocaste nada de la configuración, entrás con el confiable admin | admin. Ej: http://localhost:12345/dashboard/login
Adicionalmente hay un monitoreo por nro de endpoint (no miré como se configura esta numeración) que se ve haciendo url:puerto + "/dashboard/" + nro_endpoint + "/hourly_load" Ej: http://localhost:12345/dashboard/endpoint/1/hourly_load 


Si están desde Deepnote se pueden saltear el tema del puerto y les va a quedar algo así:

https://d0186863-7f19-4501-966b-e3e8527212f8.deepnoteproject.com/dashboard/login



![Picture title](Imagenes/image-20221102-210015.png)

# Para leer lo que guarda el dashboard por defecto

In [21]:
# Acá podemos leer el esquema y mirar una tabla de lo que fuimos logueando

con = sqlite3.connect("flask_monitoringdashboard.db")

requests_df = pd.read_sql_query("SELECT * FROM request", con)

con.close()

requests_df


,id,endpoint_id,duration,time_requested,version_requested,group_by,ip,status_code
0,1,1,9.184361,2022-10-30 23:26:39.085794,1.0,None,127.0.0.1,200
1,2,1,5.230665,2022-10-30 23:26:40.175787,1.0,None,127.0.0.1,200
2,3,1,9.117126,2022-10-30 23:26:40.726398,1.0,None,127.0.0.1,200
3,4,1,11.031866,2022-10-30 23:26:40.917601,1.0,None,127.0.0.1,200
4,5,1,12.999296,2022-10-30 23:26:41.111080,1.0,None,127.0.0.1,200
...,...,...,...,...,...,...,...,...
499,500,1,4.684210,2022-11-02 21:44:48.198757,1.0,None,172.3.133.85,200
500,501,1,4.055977,2022-11-02 21:44:59.195570,1.0,None,172.3.22.214,200
501,502,1,491.548777,2022-11-02 22:25:05.114345,1.0,None,172.3.150.40,200
502,503,1,5.240679,2022-11-02 22:25:08.965364,1.0,None,172.3.133.85,200


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d0186863-7f19-4501-966b-e3e8527212f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>